# HCPCS Data Structures from Medicare Sample

This notebook constructs HCPCS data structures to facilitate training the skip-gram model using a sample of Medicare Part B data from 2012.

A HCPCS set is an unordered list of unique HCPCS codes that occur together.

In the Medicare data set, data is aggregated by provider (NPI), HCPCS code, and year.

For each (NPI, year) pair, we will combine all HCPCS codes provided by the given provider that year. These are effectively procedure codes rendered within the same context.

In [2]:
import pandas as pd
import numpy as np
import os
import sys

## Load Medicare Data

In [4]:
data_dir = os.environ['CMS_RAW']

In [7]:
%ls $data_dir/2012

Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_CY2012.csv.gz


## Extract HCPCS Sets